In [350]:
import pandas
from enum import Enum
from IPython.display import HTML

class ReviewType(Enum):
    Discarded = -2
    Unrelated = -1
    Unreviewed = 0
    LowerDimension = 1
    Referenced = 2
    Selected = 3
    
class Constraint(Enum):
    StaticStability = "Static Stability"
    DynamicStability = "Dynamic Stability"
    PlanarRotations = "Planar Rotations"
    Verticalizations = "Verticalizations"
    UnloadPriority = "Unload Priority"
    FamilyGroupings = "Family Groupings"
    Visibility = "Visibility"
    ConcaveItems = "Concave Items"
    LoadBearing = "Load Bearing"
    LayerPacking = "Layer Packing"
    DeliveryTime = "Delivery Time"
    ContainerShape = "Container Shape"
    
class AlgorithmApproach(Enum):
    Exact = 0,
    Heuristic = 1

class Pub:
    def __init__(self, name, url, year, citations, authors, review=ReviewType.Unreviewed, notes='', summary='', isReview=False, constraints=[], algorithmName=None, algorithmType=None, algorithmApproach=None, journal=''):
        self.name = name
        self.url = url
        self.year = year
        self.citations = citations
        self.authors = authors.lower().split(",")
        self.review = review
        self.notes = notes
        self.summary = summary
        self.isReview = isReview
        self.constraints = constraints
        self.algorithmName = algorithmName
        self.algorithmType = algorithmType
        self.algorithmApproach = algorithmApproach
        self.journal = journal
        
    def to_dict(self):
        dict = {
            'name': "<a href='" + self.url + "'>" + self.name + "</a>",
            'year': self.year,
            'citations': self.citations
        }
        i = 0
        for author in self.authors:
            i += 1
            dict['author_' + str(i)] = author
        return dict;
    
    def to_review_dict(self):
        dict = {
            'name': "<a href='" + self.url + "'>" + self.name + "</a>",
            'year': self.year,
            'citations': self.citations,
            'review': self.review.value,
            'reviewState': self.review.name,
            'notes': self.notes
        }
        return dict;
    
    def to_overview_table(self):
        dict = {
            'Name': "<a href='" + self.url + "'>" + str(self.algorithmName) + "</a>",
            'Year': self.year,
            'Citations': self.citations,
            'Type': str(self.algorithmType),
            'Approach': self.algorithmApproach.name
        }
        for constraintType in Constraint:
            dict[constraintType.value] = "$\checkmark$" if constraintType in self.constraints else ""
        dict['Journal'] = self.journal
        return dict;
    
def printFrame(df):
    display(HTML(df.to_html(
        render_links=True,
        escape=False,
    )))

In [351]:
pubs = [
    Pub("An ACO Algorithm for the 3D Bin Packing Problem in the Steel Industry", 
        "https://link.springer.com/chapter/10.1007/978-3-642-38577-3_55",
        2013, 9,
        "ME Silveira, SM Vieira, JMDC Sousa"),
    
    Pub("The load-balanced multi-dimensional bin-packing problem", 
        "https://www.sciencedirect.com/science/article/abs/pii/S0305054816300909",
        2016, 41,
        "A Trivella, D Pisinger"),
    
    Pub("Metaheuristics for the 3D bin packing problem in the steel industry", 
        "https://ieeexplore.ieee.org/abstract/document/6900515",
        2014, 6,
        "JPL Viegas, SM Vieira, JMC Sousa"),
    
    Pub("Learning to Pack: A Data-Driven Tree Search Algorithm for Large-Scale 3D Bin Packing Problem", 
        "https://dl.acm.org/doi/abs/10.1145/3459637.3481933?casa_token=kxdonLrNQ4UAAAAA:0gvXOotw1hiC7xxlHXM2EAIQFyDADTuPk031hqy-8dLreYqQYLYjbxhkXFPnUhjY4E5cZDTUWPcIRg",
        2021, 0,
        "Q Zhu, X Li, Z Zhang, Z Luo, X Tong, M Yuan",
       review=ReviewType.Selected, notes="Uses convolutional neural net to prune the tree during a tree search. Same idea of super items (called blocks), accounts for support purely on area (and limits the number of supporting items allowed based on height). Used by Huawey."),
    
    Pub("A greedy search for the three-dimensional bin packing problem: the packing static stability case", 
        "https://onlinelibrary.wiley.com/doi/abs/10.1111/1475-3995.00400",
        2003, 69,
        "JL de Castro Silvaa, NY Somaa, N Maculanb"),
    
    Pub("Optimizing three-dimensional bin packing through simulation", 
        "https://www.academia.edu/download/55848730/Optimizaing_3-dimensional_bin_packing_through_simulation.pdf",
        2006, 11,
        "E Dube, LR Kanavathy, P Woodview"),
    
    Pub("Multi-objective 3D bin-packing problem", 
        "https://ieeexplore.ieee.org/abstract/document/8880442",
        2019, 3,
        "J Hasan, J Kaabi, Y Harrath"),
    
    Pub("An Approximation Scheme for Bin Packing with Conflicts", 
        "https://link.springer.com/article/10.1023/A:1009871302966",
        1999, 103,
        "K Jansen"),
    
    Pub("A three-stage layer-based heuristic to solve the 3D bin-packing problem under balancing constraint", 
        "https://www.sciencedirect.com/science/article/pii/S1319157821001749",
        2021, 0,
        "Y Harrath",
        review=ReviewType.Selected, notes="Load balancing and some really simple instances to test against. Layer based, without explicit support check. Similar superitems for layer generation (with verticalizations + rotations)."),
    
    Pub("3D heterogeneous bin packing framework for multi-constrained problems using hybrid genetic approach", 
        "https://iopscience.iop.org/article/10.1088/1757-899X/402/1/012203/meta",
        2018, 4,
        "SKR Kanna, KC Udaiyakumar"),
    
    Pub("A Hybrid Genetic Algorithm for Packing in 3D with Deepest Bottom Left with Fill Method", 
        "https://link.springer.com/chapter/10.1007/978-3-540-30198-1_45",
        2004, 69,
        "K Karabulut, MM İnceoğlu"),
    
    Pub("A Framework for Robotic Bin Packing with a Dual-Arm Configuration", 
        "https://link.springer.com/chapter/10.1007/978-3-030-20131-9_277",
        2019, 6,
        "CY Weng, W Yin, ZJ Lim, IM Chen"),
    
    Pub("A heuristic for solving large bin packing problems in two and three dimensions", 
        "https://link.springer.com/article/10.1007/s10100-010-0184-1",
        2012, 26,
        "D Mack, A Bortfeldt"),
    
    Pub("A mixed integer programming formulation for the three-dimensional bin packing problem deriving from an air cargo application", 
        "https://onlinelibrary.wiley.com/doi/abs/10.1111/itor.12111",
        2016, 83,
        "C Paquay, M Schyns, S Limbourg",
        notes="Nice problem formulation and table comparing various algorithms", review=ReviewType.Selected,
        algorithmName="Paquay, Schyns, Limbourg", algorithmType="MILP", algorithmApproach=AlgorithmApproach.Exact,
        constraints=[Constraint.ContainerShape, Constraint.PlanarRotations, Constraint.Verticalizations, Constraint.StaticStability, Constraint.LoadBearing, Constraint.DynamicStability],
        journal="International Transactions in Operational Research 23 (2016)"),
    
    Pub("Smart Packing Simulator for 3D Packing Problem Using Genetic Algorithm", 
        "https://iopscience.iop.org/article/10.1088/1742-6596/1447/1/012041/meta",
        2020, 4,
        "U Khairuddin, N Razi, MSZ Abidin",
        review=ReviewType.Discarded, notes="No stability, mostly educational, BRKGA is a more interesting approach"),
    
    Pub("Hybrid greedy heuristics based on linear programming for the three-dimensional single bin-size bin packing problem", 
        "https://onlinelibrary.wiley.com/doi/abs/10.1111/itor.12035",
        2014, 29,
        "M Hifi, S Negre, L Wu"),
    
    Pub("Jampacker: An Efficient and Reliable Robotic Bin Packing System for Cuboid Objects", 
        "https://ieeexplore.ieee.org/abstract/document/9286471",
        2020, 6,
        "M Agarwal, S Biswas, C Sarkar, S Paul"),
    
    Pub("On the online bin packing problem", 
        "https://dl.acm.org/doi/abs/10.1145/585265.585269?casa_token=v7qy7d0Jf4MAAAAA:HNbmNu1NxPVKE2-mU1WsLtHjzcSzwdPZUleMuqBfWRaxwdvTX9lpp76cIYqQupRORYpgk1JNIhDerw",
        2002, 338,
        "SS Seiden"),
    
    Pub("Algorithms for the Bin Packing Problem with Conflicts", 
        "https://pubsonline.informs.org/doi/abs/10.1287/ijoc.1090.0355",
        2010, 127,
        "AEF Muritiba, M Iori, E Malaguti"),
    
    Pub("Three-dimensional bin packing problem with variable bin height", 
        "https://www.sciencedirect.com/science/article/pii/S0377221709003919",
        2010, 132,
        "Y Wu, W Li, M Goh, R de Souza"),
    
    Pub("Algorithms for the variable sized bin packing problem", 
        "https://www.sciencedirect.com/science/article/pii/S0377221702002473",
        2003, 213,
        "J Kang, S Park"),
    
    Pub("Heuristic algorithms for the three-dimensional bin packing problem", 
        "https://www.sciencedirect.com/science/article/pii/S0377221702001340",
        2002, 229,
        "A Lodi, S Martello, D Vigo"),
    
    Pub("Extreme Point-Based Heuristics for Three-Dimensional Bin Packing", 
        "https://pubsonline.informs.org/doi/abs/10.1287/ijoc.1070.0250",
        2008, 220,
        "TG Crainic, G Perboli, R Tadei"),
    
    Pub("Approximation and online algorithms for multidimensional bin packing: A survey", 
        "https://www.sciencedirect.com/science/article/pii/S1574013716301356",
        2017, 130,
        "HI Christensen, A Khan, S Pokutta, P Tetali"),
    
    Pub("A hybrid genetic algorithm with a new packing strategy for the three-dimensional bin packing problem", 
        "https://www.sciencedirect.com/science/article/pii/S0096300312007369",
        2012, 56,
        "K Kang, I Moon, H Wang"),
    
    Pub("The generalized bin packing problem", 
        "https://www.sciencedirect.com/science/article/pii/S1366554512000580",
        2012, 52,
        "MM Baldi, TG Crainic, G Perboli, R Tadei"),
    
    Pub("Guided Local Search for the Three-Dimensional Bin-Packing Problem", 
        "https://pubsonline.informs.org/doi/abs/10.1287/ijoc.15.3.267.16080",
        2003, 257,
        "O Faroe, D Pisinger"),
    
    Pub("LP models for bin packing and cutting stock problems", 
        "https://www.sciencedirect.com/science/article/pii/S0377221702001248",
        2002, 327,
        "JMV De Carvalho"),
    
    Pub("Evolving Bin Packing Heuristics with Genetic Programming", 
        "https://link.springer.com/chapter/10.1007/11844297_87",
        2006, 168,
        "EK Burke, MR Hyde, G Kendall"),
    
    Pub("Heuristics and lower bounds for the bin packing problem with conflicts", 
        "https://www.sciencedirect.com/science/article/pii/S0305054802001958",
        2004, 138,
        "M Gendreau, G Laporte, F Semet"),
    
    Pub("Selfish Bin Packing", 
        "https://link.springer.com/article/10.1007/s00453-009-9348-6",
        2011, 48,
        "L Epstein, E Kleiman"),
    
    Pub("Heuristics for the variable sized bin-packing problem", 
        "https://www.sciencedirect.com/science/article/pii/S0305054808002748",
        2009, 114,
        "M Haouari, M Serairi"),
    
    Pub("Ant colony optimization and local search for bin packing and cutting stock problems", 
        "https://www.tandfonline.com/doi/abs/10.1057/palgrave.jors.2601771",
        2004, 279,
        "J Levine, F Ducatelle"),
    
    Pub("TS2PACK: A two-level tabu search for the three-dimensional bin packing problem", 
        "https://www.sciencedirect.com/science/article/pii/S0377221707010995",
        2009, 146,
        "TG Crainic, G Perboli, R Tadei", review=ReviewType.Selected,
        algorithmName="TS2Pack", algorithmType="Tabu", algorithmApproach=AlgorithmApproach.Heuristic,
        constraints=[],
        journal="European Journal of Operational Research 195 (2009)"),
    
    Pub("Online bin packing with advice", 
        "https://link.springer.com/content/pdf/10.1007/s00453-014-9955-8.pdf",
        2016, 59,
        "J Boyar, S Kamali, KS Larsen, A López-Ortiz"),
    
    Pub("Bin packing and cutting stock problems: Mathematical models and exact algorithms", 
        "https://www.sciencedirect.com/science/article/pii/S0377221716302491",
        2016, 245,
        "M Delorme, M Iori, S Martello",
        isReview=True, review=ReviewType.LowerDimension),
    
    Pub("A Linear Programming Approach for the Three-Dimensional Bin-Packing Problem", 
        "https://www.sciencedirect.com/science/article/pii/S1571065310001277",
        2010, 64,
        "M Hifi, I Kacem, S Nègre, L Wu"),
    
    Pub("On solving multiobjective bin packing problems using evolutionary particle swarm optimization", 
        "https://www.sciencedirect.com/science/article/pii/S037722170700567X",
        2008, 170,
        "DS Liu, KC Tan, SY Huang, CK Goh, WK Ho"),
    
    Pub("A Set-Covering-Based Heuristic Approach for Bin-Packing Problems", 
        "https://pubsonline.informs.org/doi/abs/10.1287/ijoc.1040.0089",
        2006, 108,
        "M Monaci, P Toth"),
    
    Pub("Hyper-heuristics: learning to combine simple heuristics in bin-packing problems", 
        "http://gpbib.pmacs.upenn.edu/gecco2002/LCS260.pdf",
        2002, 189,
        "P Ross, S Schulenburg"),
    
    Pub("Bin Packing with Conflicts: A Generic Branch-and-Price Algorithm", 
        "https://pubsonline.informs.org/doi/abs/10.1287/ijoc.1120.0499",
        2013, 119,
        "R Sadykov, F Vanderbeck"),
    
    Pub("Polynomiality for Bin Packing with a Constant Number of Item Types", 
        "https://epubs.siam.org/doi/abs/10.1137/1.9781611973402.61",
        2014, 77,
        "MX Goemans, T Rothvoß"),
    
    Pub("New lower bounds for certain classes of bin packing algorithms", 
        "https://www.sciencedirect.com/science/article/pii/S0304397512003611",
        2012, 129,
        "J Balogh, J Békési, G Galambos"),
    
    Pub("A hybrid GRASP/VND algorithm for two- and three-dimensional bin packing", 
        "https://link.springer.com/article/10.1007/s10479-008-0449-4",
        2010, 92,
        "F Parreño, R Alvarez-Valdés, JF Oliveira"),
    
    Pub("A Novel Quantum Inspired Cuckoo Search Algorithm for Bin Packing Problem", 
        "http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.471.8615&rep=rep1&type=pdf",
        2012, 73,
        "A Layeb, SR Boussalia"),
    
    Pub("New and improved level heuristics for the rectangular strip packing and variable-sized bin packing problems", 
        "https://www.sciencedirect.com/science/article/pii/S0377221709005360",
        2010, 86,
        "FG Ortmann, N Ntene, JH Van Vuuren"),
    
    Pub("Constraints in container loading–A state-of-the-art review", 
        "https://www.sciencedirect.com/science/article/pii/S037722171200937X",
        2013, 375,
        "A Bortfeldt, G Wäscher"),
    
    Pub("Recent advances in multi-dimensional packing problems", 
        "https://core.ac.uk/download/pdf/234885840.pdf",
        2012, 33,
        "T Crainic, G Perboli, R Tadei"),
    
    Pub("Exact solution techniques for two-dimensional cutting and packing", 
        "https://www.sciencedirect.com/science/article/pii/S0377221720306111",
        2021, 11,
        "M Iori, VL de Lima, S Martello, FK Miyazawa",
        review=ReviewType.LowerDimension),
    
    Pub("A grouping genetic algorithm with controlled gene transmission for the bin packing problem", 
        "https://www.sciencedirect.com/science/article/pii/S0305054814002676",
        2015, 91,
        "M Quiroz-Castellanos, L Cruz-Reyes"),
    
    Pub("A biased random key genetic algorithm for 2D and 3D bin packing problems", 
        "https://www.sciencedirect.com/science/article/pii/S0925527313001837",
        2013, 138,
        "JF Gonçalves, MGC Resende",
        review=ReviewType.Selected),
    
    Pub("Class constrained bin packing revisited", 
        "https://www.sciencedirect.com/science/article/pii/S0304397510002549",
        2010, 43,
        "L Epstein, C Imreh, A Levin"),
    
    Pub("The bin packing problem with precedence constraints", 
        "https://pubsonline.informs.org/doi/abs/10.1287/opre.1120.1109",
        2012, 33,
        "M Dell'Amico, JCD Díaz, M Iori"),
    
    Pub("A branch-and-price algorithm for the bin packing problem with conflicts", 
        "https://pubsonline.informs.org/doi/abs/10.1287/ijoc.1100.0406",
        2011, 48,
        "S Elhedhli, L Li, M Gzara"),
    
    Pub("Bin packing and related problems: General arc-flow formulation with graph compression", 
        "https://www.sciencedirect.com/science/article/pii/S0305054815002762",
        2016, 89,
        "F Brandao, JP Pedroso"),
    
    Pub("A particular approach for the Three-dimensional Packing Problem with additional constraints", 
        "https://www.sciencedirect.com/science/article/pii/S0305054810000213",
        2010, 34,
        "A de Almeida, MB Figueiredo"),
    
    Pub("A hybrid genetic algorithm for 3D bin packing problems", 
        "https://ieeexplore.ieee.org/abstract/document/5645211/",
        2010, 28,
        "H Wang, Y Chen"),
    
    Pub("An efficient metaheuristic for multi-dimensional multi-container packing", 
        "https://ieeexplore.ieee.org/abstract/document/6042476/",
        2011, 28,
        "G Perboli, TG Crainic, R Tadei"),
    
    Pub("A constructive heuristic for the three dimensional Bin Packing Problem with transportation constraints", 
        "https://orbi.uliege.be/handle/2268/171198",
        2014, 0,
        "C Paquay, M Schyns, S Limbourg"),
    
    Pub("The three-dimensional bin packing problem", 
        "https://pubsonline.informs.org/doi/abs/10.1287/opre.48.2.256.12386",
        2000, 741,
        "S Martello, D Pisinger, D Vigo", notes="Exact approach, cited by almost every paper so far", review=ReviewType.Selected,
        algorithmName="S. Martello, D. Pisinger, D. Vigo", algorithmType="B&B", algorithmApproach=AlgorithmApproach.Exact,
        constraints=[Constraint.LayerPacking],
        journal="Operations Research 48 (2000)"),
    
    Pub("A new heuristic algorithm for the 3D bin packing problem", 
        "https://link.springer.com/chapter/10.1007/978-1-4020-8735-6_64",
        2008, 10,
        "WF Maarouf, AM Barbar, MJ Owayjan"),
    
    Pub("Space defragmentation heuristic for 2D and 3D bin packing problems", 
        "https://www.aaai.org/ocs/index.php/IJCAI/IJCAI11/paper/viewPaper/3263",
        2011, 10,
        "Z Zhang, S Guo, W Zhu, WC Oon, A Lim"),
    
    Pub("Optimization of heterogeneous Bin packing using adaptive genetic algorithm", 
        "https://iopscience.iop.org/article/10.1088/1757-899X/183/1/012026/meta",
        2017, 25,
        "R Sridhar, M Chandrasekaran"),
    
    Pub("Solving 3D Bin Packing Problem via Multimodal Deep Reinforcement Learning", 
        "http://www.ifaamas.org/Proceedings/aamas2021/pdfs/p1548.pdf",
        2021, 1,
        "Y Jiang, Z Cao, J Zhang",
        review=ReviewType.Discarded, notes="Only accounts for packing without any practical constraint"),
    
    Pub("Quasi-Monte-Carlo Tree Search for 3D Bin Packing", 
        "https://link.springer.com/chapter/10.1007/978-3-030-03398-9_33",
        2018, 3,
        "H Li, Y Wang, DP Ma, Y Fang, Z Lei"),
    
    Pub("A Fuzzy Logic Classifier for the Three Dimensional Bin Packing Problem Deriving From Package Delivery Companies Application", 
        "https://www.igi-global.com/chapter/a-fuzzy-logic-classifier-for-the-three-dimensional-bin-packing-problem-deriving-from-package-delivery-companies-application/227182",
        2019, 1,
        "PH Hernández, N Castillo-García"),
    
    Pub("A Generalized Reinforcement Learning Algorithm for Online 3D Bin-Packing", 
        "https://arxiv.org/abs/2007.00463",
        2020, 6,
        "R Verma, A Singhal, H Khadilkar, A Basumatary"),
    
    Pub("THREE-DIMENSIONAL BIN PACKING PROBLEM WITH A STABILITY REJECTION CRITERION", 
        "https://www.eccomas2016.org/proceedings/pdf/13092.pdf",
        2016, 1,
        "T Linkosaari, T Urponen, H Juvonen",
        notes="Interesting stability constraint"),
    
    Pub("A Multi-task Selected Learning Approach for Solving 3D Flexible Bin Packing Problem", 
        "https://arxiv.org/abs/1804.06896",
        2018, 22,
        "L Duan, H Hu, Y Qian, Y Gong, X Zhang, Y Xu"),
    
    Pub("Mathematical models and heuristic algorithms for pallet building problems with practical constraints", 
        "https://link.springer.com/article/10.1007/s10479-021-04349-w",
        2021, 0,
        "G Calzavara, M Iori, M Locatelli, M CO Moreira, T Silveira",
        review=ReviewType.Selected,
        notes="Cites books that treat the subject of bin packing, handles stability, visibility and contiguity constraints in container loading. They build pallets with extreme points (with additionsl points for visibility and contiguity) and then stack them with Reactive GRASP. Further expansion of these is provided by introducing mathematical models to both phases.",
        algorithmName="Calzavara, Iori et al.", algorithmType="(Constructive Heuristic/MILP)+GRASP", algorithmApproach=AlgorithmApproach.Heuristic,
        constraints=[Constraint.LayerPacking, Constraint.StaticStability, Constraint.LoadBearing, Constraint.Visibility, Constraint.FamilyGroupings, Constraint.PlanarRotations],
        journal="Annals of Operations Research (2021)"),
    
    Pub("The pallet loading problem: a review of solution methods and computational experiments",
        "https://onlinelibrary.wiley.com/doi/abs/10.1111/itor.12099",
        2016, 33,
        "E Silva, JF Oliveira, G Waescher"),
    
    Pub("The exact solutions of several types of container loading problems",
        "https://www.sciencedirect.com/science/article/pii/S0377221719310136",
        2020, 14,
        "DV Kurpel, CT Scarpin, JEP Junior",
        review=ReviewType.Selected),
    
    Pub("Heuristics for container loading of furniture",
        "https://www.sciencedirect.com/science/article/pii/S0377221709000563",
        2012, 71,
        "J Egeblad, C Garavelli, S Lisi, D Pisinger",
        review=ReviewType.Unrelated,
        notes="Treat shapes instead of boxes, not really comparable. Local search to improve stability before removing the solutions (idea of temporarly unstable solutions that can increase efficiency)",
        algorithmName="Egeblad, Garavelli, Lisi, Pisinger", algorithmType="Tree Search+ Local Serach+ Greedy Search", algorithmApproach=AlgorithmApproach.Heuristic,
        constraints=[Constraint.PlanarRotations, Constraint.Verticalizations, Constraint.StaticStability, Constraint.ConcaveItems, Constraint.LoadBearing],
        journal="European Journal of Operational Research 200 (2010)"),
    
    Pub("A GRASP algorithm for multi container loading problems with practical constraints",
        "https://link.springer.com/article/10.1007/s10288-018-0397-z",
        2020, 5,
        "MT Alonso, R Alvarez-Valdés, F Parreño",
        ReviewType.Unrelated, notes="Dynamic and Static stability in truck loading with center o mass on axles. GRASP technique. Finds suitable positions in the container, calculates parameters for suitable pallets in given position, stacks layers into pallets with static stability. Pallets are of different kind (in our case mostly highly etherogeneous, in their they try to avoid etherogeneous pallets)",
        algorithmName="Alonso, Alvarez-Valdés, Parreño et al.", algorithmType="GRASP", algorithmApproach=AlgorithmApproach.Heuristic,
        constraints=[Constraint.PlanarRotations, Constraint.Verticalizations, Constraint.LayerPacking, Constraint.StaticStability, Constraint.DynamicStability, Constraint.LoadBearing, Constraint.DeliveryTime],
        journal="4OR - A Quarterly Journal of Operations Research 18 (2020)"),
    
    Pub("The fixed set search applied to the power dominating set problem",
        "https://onlinelibrary.wiley.com/doi/abs/10.1111/exsy.12559",
        2020, 8,
        "R Jovanovic, S Voss",
        review=ReviewType.Unrelated, notes="Cited as an interesting future development"),
    
    Pub("BRKGA/VND hybrid algorithm for the classic three-dimensional bin packing problem",
       "https://www.sciencedirect.com/science/article/pii/S1571065318300696",
       2018, 12,
       "A Zudio, DH da Silva Costa, BP Masquio",
       review=ReviewType.Selected, notes="Improvement over BRKGA by using a constructive greedy heristic to give an order for BRKGA. Provides tests on standard instances by Martello 2000 and compared to other heuristics (excluding Extreme Points)",
       algorithmName="A Zudio et al.", algorithmType="BRKGA+VND", algorithmApproach=AlgorithmApproach.Heuristic,
       constraints=[Constraint.PlanarRotations, Constraint.Verticalizations],
       journal="Electronic Notes in Discrete Mathematics 66 (2018)")
]

for pub in pubs:
    for other in pubs:
        if not(pub == other) and pub.name.lower()==other.name.lower():
            raise Exception('Duplicate entry "' + pub.name + '"')

# Overview

In [352]:
import numpy as np
df = pandas.DataFrame.from_records([pub.to_dict() for pub in pubs])

print("Publications: " + str(len(df.index)))
with pandas.option_context("display.max_rows", len(df.index)):
    printFrame(df.sort_values(['year', 'citations'], ascending=[False, False]).replace(np.nan, ""))

Publications: 76


,name,year,citations,author_1,author_2,author_3,author_4,author_5,author_6
48,Exact solution techniques for two-dimensional cutting and packing,2021,11,m iori,vl de lima,s martello,fk miyazawa,,
63,Solving 3D Bin Packing Problem via Multimodal Deep Reinforcement Learning,2021,1,y jiang,z cao,j zhang,,,
3,Learning to Pack: A Data-Driven Tree Search Algorithm for Large-Scale 3D Bin Packing Problem,2021,0,q zhu,x li,z zhang,z luo,x tong,m yuan
8,A three-stage layer-based heuristic to solve the 3D bin-packing problem under balancing constraint,2021,0,y harrath,,,,,
69,Mathematical models and heuristic algorithms for pallet building problems with practical constraints,2021,0,g calzavara,m iori,m locatelli,m co moreira,t silveira,
71,The exact solutions of several types of container loading problems,2020,14,dv kurpel,ct scarpin,jep junior,,,
74,The fixed set search applied to the power dominating set problem,2020,8,r jovanovic,s voss,,,,
16,Jampacker: An Efficient and Reliable Robotic Bin Packing System for Cuboid Objects,2020,6,m agarwal,s biswas,c sarkar,s paul,,
66,A Generalized Reinforcement Learning Algorithm for Online 3D Bin-Packing,2020,6,r verma,a singhal,h khadilkar,a basumatary,,
73,A GRASP algorithm for multi container loading problems with practical constraints,2020,5,mt alonso,r alvarez-valdés,f parreño,,,


## GroupBy Author Citations

In [353]:
authorMap = {}
for pub in pubs:
    for author in pub.authors:
        if author in authorMap:
            authorMap[author] = authorMap[author] + pub.citations
        else:
            authorMap[author] = pub.citations
with pandas.option_context("display.max_rows", len(authorMap)):
    printFrame(pandas.DataFrame.from_dict(authorMap, orient='index').sort_values(0, ascending=False))







,0
d pisinger,1110
d vigo,970
s martello,741
s martello,485
r tadei,479
g perboli,451
m iori,405
a bortfeldt,375
g wäscher,375
tg crainic,366


In [354]:
import datetime
nAuthorMap = {}
minYear = 1999

def normalizedCitations(citations, year):
    return (max(0, (year - minYear))/(datetime.datetime.now().year - minYear)) * citations

for pub in pubs:
    for author in pub.authors:
        if author in nAuthorMap:
            nAuthorMap[author] = nAuthorMap[author] + normalizedCitations(pub.citations, pub.year)
        else:
            nAuthorMap[author] = normalizedCitations(pub.citations, pub.year)
with pandas.option_context("display.max_rows", len(nAuthorMap)):
    printFrame(pandas.DataFrame.from_dict(nAuthorMap, orient='index').sort_values(0, ascending=False))

,0
m iori,272.318182
g wäscher,238.636364
a bortfeldt,238.636364
s martello,231.545455
r tadei,221.863636
g perboli,206.590909
m delorme,189.318182
tg crainic,156.363636
d pisinger,154.045455
hi christensen,106.363636


## To Review Strategy

In [355]:
import numpy as np

#We can sort pubblications based on the "relevance" of its best authors
bestNormalizedAuthorScore = 0

for author in nAuthorMap:
    if nAuthorMap[author] > bestNormalizedAuthorScore: bestNormalizedAuthorScore = nAuthorMap[author]
        
def scorePubblication(pub, pubDict):
    mostRelevantAuthorScore = 0
    for author in pub.authors:
        if nAuthorMap[author] > mostRelevantAuthorScore: mostRelevantAuthorScore = nAuthorMap[author]
    score = normalizedCitations(pub.citations, pub.year) * (mostRelevantAuthorScore / bestNormalizedAuthorScore)
    pubDict["citations"] = mostRelevantAuthorScore + score
    return pubDict

#Then we can filter out the pubblications we already reviewed
toReviewPub = [scorePubblication(pub, pub.to_dict()) for pub in pubs if pub.review == ReviewType.Unreviewed]
df = pandas.DataFrame.from_records(toReviewPub)

print("To review: " + str(len(df.index)))

To review: 60


### To Review by Score

In [356]:
with pandas.option_context('display.max_colwidth', None, 
                           "display.max_rows", len(df.index)):
    printFrame(df.sort_values(['citations'], ascending=[False]).replace(np.nan, ""))

,name,year,citations,author_1,author_2,author_3,author_4,author_5,author_6
40,Constraints in container loading–A state-of-the-art review,2013,447.756863,a bortfeldt,g wäscher,,,,
14,Algorithms for the Bin Packing Problem with Conflicts,2010,335.818182,aef muritiba,m iori,e malaguti,,,
18,Extreme Point-Based Heuristics for Three-Dimensional Bin Packing,2008,295.188624,tg crainic,g perboli,r tadei,,,
44,The bin packing problem with precedence constraints,2012,291.818182,m dell'amico,jcd díaz,m iori,,,
17,Heuristic algorithms for the three-dimensional bin packing problem,2002,258.097237,a lodi,s martello,d vigo,,,
21,The generalized bin packing problem,2012,246.897824,mm baldi,tg crainic,g perboli,r tadei,,
41,Recent advances in multi-dimensional packing problems,2012,237.750717,t crainic,g perboli,r tadei,,,
49,An efficient metaheuristic for multi-dimensional multi-container packing,2011,234.306665,g perboli,tg crainic,r tadei,,,
22,Guided Local Search for the Three-Dimensional Bin-Packing Problem,2003,180.478225,o faroe,d pisinger,,,,
1,The load-balanced multi-dimensional bin-packing problem,2016,171.967284,a trivella,d pisinger,,,,


### To Review by Score divided in years

In [357]:
with pandas.option_context('display.max_colwidth', None, 
                           "display.max_rows", len(df.index)):
    printFrame(df.sort_values(['year', 'citations'], ascending=[False, False]).replace(np.nan, ""))

,name,year,citations,author_1,author_2,author_3,author_4,author_5,author_6
12,Jampacker: An Efficient and Reliable Robotic Bin Packing System for Cuboid Objects,2020,5.847726,m agarwal,s biswas,c sarkar,s paul,,
56,A Generalized Reinforcement Learning Algorithm for Online 3D Bin-Packing,2020,5.847726,r verma,a singhal,h khadilkar,a basumatary,,
9,A Framework for Robotic Bin Packing with a Dual-Arm Configuration,2019,5.563800,cy weng,w yin,zj lim,im chen,,
5,Multi-objective 3D bin-packing problem,2019,2.754586,j hasan,j kaabi,y harrath,,,
55,A Fuzzy Logic Classifier for the Three Dimensional Bin Packing Problem Deriving From Package Delivery Companies Application,2019,0.912126,ph hernández,n castillo-garcía,,,,
58,A Multi-task Selected Learning Approach for Solving 3D Flexible Bin Packing Problem,2018,20.325655,l duan,h hu,y qian,y gong,x zhang,y xu
7,3D heterogeneous bin packing framework for multi-constrained problems using hybrid genetic approach,2018,3.498369,skr kanna,kc udaiyakumar,,,,
54,Quasi-Monte-Carlo Tree Search for 3D Bin Packing,2018,2.615560,h li,y wang,dp ma,y fang,z lei,
19,Approximation and online algorithms for multidimensional bin packing: A survey,2017,147.907771,hi christensen,a khan,s pokutta,p tetali,,
53,Optimization of heterogeneous Bin packing using adaptive genetic algorithm,2017,21.990941,r sridhar,m chandrasekaran,,,,


# Review Results

In [358]:
scoredPubs = [scorePubblication(pub, pub.to_review_dict()) for pub in pubs if not pub.review == ReviewType.Unreviewed]
df = pandas.DataFrame.from_records(scoredPubs)

with pandas.option_context('display.max_colwidth', None, 
                           "display.max_rows", len(df.index)):
    printFrame(df.sort_values(['review', 'citations'], ascending=[False, False]).replace(np.nan, ""))

,name,year,citations,review,reviewState,notes
4,TS2PACK: A two-level tabu search for the three-dimensional bin packing problem,2009,275.931556,3,Selected,
10,Mathematical models and heuristic algorithms for pallet building problems with practical constraints,2021,272.318182,3,Selected,"Cites books that treat the subject of bin packing, handles stability, visibility and contiguity constraints in container loading. They build pallets with extreme points (with additionsl points for visibility and contiguity) and then stack them with Reactive GRASP. Further expansion of these is provided by introducing mathematical models to both phases."
8,The three-dimensional bin packing problem,2000,173.098648,3,Selected,"Exact approach, cited by almost every paper so far"
7,A biased random key genetic algorithm for 2D and 3D bin packing problems,2013,116.138116,3,Selected,
2,A mixed integer programming formulation for the three-dimensional bin packing problem deriving from an air cargo application,2016,79.241757,3,Selected,Nice problem formulation and table comparing various algorithms
11,The exact solutions of several types of container loading problems,2020,14.019438,3,Selected,
15,BRKGA/VND hybrid algorithm for the classic three-dimensional bin packing problem,2018,10.758046,3,Selected,Improvement over BRKGA by using a constructive greedy heristic to give an order for BRKGA. Provides tests on standard instances by Martello 2000 and compared to other heuristics (excluding Extreme Points)
0,Learning to Pack: A Data-Driven Tree Search Algorithm for Large-Scale 3D Bin Packing Problem,2021,0.000000,3,Selected,"Uses convolutional neural net to prune the tree during a tree search. Same idea of super items (called blocks), accounts for support purely on area (and limits the number of supporting items allowed based on height). Used by Huawey."
1,A three-stage layer-based heuristic to solve the 3D bin-packing problem under balancing constraint,2021,0.000000,3,Selected,"Load balancing and some really simple instances to test against. Layer based, without explicit support check. Similar superitems for layer generation (with verticalizations + rotations)."
5,Bin packing and cutting stock problems: Mathematical models and exact algorithms,2016,461.636364,1,LowerDimension,


## Standard instance results with support rules, perimeter filling and spacing
Classes from I to VIII with $n \in \{50, 100, 150, 200\}$
```
[RESULT] Running Class I
[RESULT] n=50	avg_time=188.9ms	avg_bins:15.4
[RESULT] n=100	avg_time=454.5ms	avg_bins:26.3
[RESULT] n=150	avg_time=923.0ms	avg_bins:37.8
[RESULT] n=200	avg_time=1710.1ms	avg_bins:65.2

[RESULT] Running Class II
[RESULT] n=50	avg_time=206.3ms	avg_bins:17.5
[RESULT] n=100	avg_time=700.9ms	avg_bins:22.8
[RESULT] n=150	avg_time=1472.4ms	avg_bins:37.6
[RESULT] n=200	avg_time=2776.0ms	avg_bins:77.6

[RESULT] Running Class III
[RESULT] n=50	avg_time=108.3ms	avg_bins:15.1
[RESULT] n=100	avg_time=424.4ms	avg_bins:30.3
[RESULT] n=150	avg_time=922.5ms	avg_bins:44.4
[RESULT] n=200	avg_time=1616.2ms	avg_bins:69.1

[RESULT] Running Class IV
[RESULT] n=50	avg_time=134.2ms	avg_bins:30.5
[RESULT] n=100	avg_time=562.1ms	avg_bins:64.9
[RESULT] n=150	avg_time=1354.6ms	avg_bins:97.4
[RESULT] n=200	avg_time=2377.9ms	avg_bins:129.8

[RESULT] Running Class V
[RESULT] n=50	avg_time=126.6ms	avg_bins:8.5
[RESULT] n=100	avg_time=451.1ms	avg_bins:18.8
[RESULT] n=150	avg_time=960.0ms	avg_bins:29.7
[RESULT] n=200	avg_time=2691.1ms	avg_bins:30.7

[RESULT] Running Class VI
[RESULT] n=50	avg_time=147.1ms	avg_bins:22.6
[RESULT] n=100	avg_time=536.5ms	avg_bins:43.9
[RESULT] n=150	avg_time=1195.3ms	avg_bins:71.5
[RESULT] n=200	avg_time=2090.4ms	avg_bins:93.6

[RESULT] Running Class VII
[RESULT] n=50	avg_time=213.3ms	avg_bins:20.3
[RESULT] n=100	avg_time=824.4ms	avg_bins:42.5
[RESULT] n=150	avg_time=1750.9ms	avg_bins:64.5
[RESULT] n=200	avg_time=3030.8ms	avg_bins:80.2

[RESULT] Running Class VIII
[RESULT] n=50	avg_time=215.3ms	avg_bins:25.5
[RESULT] n=100	avg_time=757.4ms	avg_bins:50.2
[RESULT] n=150	avg_time=1643.1ms	avg_bins:76.6
[RESULT] n=200	avg_time=2797.7ms	avg_bins:96.7
```

## Standard Instances without support rule, no spacing and no perimeter filling
Classes from I to VIII with $n \in \{50, 100, 150, 200\}$
```
[RESULT] Running Class I
[RESULT] n=50	avg_time=63.3ms	avg_bins:15.0
[RESULT] n=100	avg_time=103.3ms	avg_bins:30.7
[RESULT] n=150	avg_time=201.1ms	avg_bins:34.0
[RESULT] n=200	avg_time=345.3ms	avg_bins:54.5

[RESULT] Running Class II
[RESULT] n=50	avg_time=61.4ms	avg_bins:15.3
[RESULT] n=100	avg_time=212.5ms	avg_bins:30.9
[RESULT] n=150	avg_time=399.9ms	avg_bins:42.3
[RESULT] n=200	avg_time=643.6ms	avg_bins:62.2

[RESULT] Running Class III
[RESULT] n=50	avg_time=28.8ms	avg_bins:17.4
[RESULT] n=100	avg_time=106.9ms	avg_bins:28.5
[RESULT] n=150	avg_time=191.3ms	avg_bins:33.9
[RESULT] n=200	avg_time=336.8ms	avg_bins:57.2

[RESULT] Running Class IV
[RESULT] n=50	avg_time=60.4ms	avg_bins:33.9
[RESULT] n=100	avg_time=174.6ms	avg_bins:68.1
[RESULT] n=150	avg_time=321.1ms	avg_bins:92.3
[RESULT] n=200	avg_time=522.5ms	avg_bins:131.7

[RESULT] Running Class V
[RESULT] n=50	avg_time=35.1ms	avg_bins:10.3
[RESULT] n=100	avg_time=115.8ms	avg_bins:14.9
[RESULT] n=150	avg_time=286.7ms	avg_bins:26.7
[RESULT] n=200	avg_time=484.4ms	avg_bins:34.4

[RESULT] Running Class VI
[RESULT] n=50	avg_time=51.8ms	avg_bins:24.6
[RESULT] n=100	avg_time=142.5ms	avg_bins:44.4
[RESULT] n=150	avg_time=296.1ms	avg_bins:67.1
[RESULT] n=200	avg_time=485.2ms	avg_bins:95.4

[RESULT] Running Class VII
[RESULT] n=50	avg_time=54.0ms	avg_bins:20.8
[RESULT] n=100	avg_time=184.6ms	avg_bins:42.1
[RESULT] n=150	avg_time=406.0ms	avg_bins:65.0
[RESULT] n=200	avg_time=696.4ms	avg_bins:87.4

[RESULT] Running Class VIII
[RESULT] n=50	avg_time=53.5ms	avg_bins:24.9
[RESULT] n=100	avg_time=168.5ms	avg_bins:51.2
[RESULT] n=150	avg_time=373.1ms	avg_bins:74.0
[RESULT] n=200	avg_time=639.5ms	avg_bins:103.4

```

## Latest Developments (Invalid)
Classes from I to IV with $n \in \{50, 100, 150\}$
```
[RESULT] Running Class I
[RESULT] n=50	avg_time=842.2ms	avg_bins:11.6
[RESULT] n=100	avg_time=2221.6ms	avg_bins:19.4
[RESULT] n=150	avg_time=5024.6ms	avg_bins:32.4

[RESULT] Running Class II
[RESULT] n=50	avg_time=506.0ms	avg_bins:11.6
[RESULT] n=100	avg_time=1871.2ms	avg_bins:21.9
[RESULT] n=150	avg_time=6447.1ms	avg_bins:41.6

[RESULT] Running Class III
[RESULT] n=50	avg_time=359.0ms	avg_bins:12.7
[RESULT] n=100	avg_time=2951.7ms	avg_bins:20.7
[RESULT] n=150	avg_time=8787.1ms	avg_bins:36.6

[RESULT] Running Class IV
[RESULT] n=50	avg_time=572.0ms	avg_bins:28.5
[RESULT] n=100	avg_time=2301.7ms	avg_bins:62.7
[RESULT] n=150	avg_time=7862.2ms	avg_bins:77.9

```

# Overview Table

In [359]:
pubsAlgorithmOverview = [pub.to_overview_table() for pub in pubs if not pub.algorithmName == None]
df = pandas.DataFrame.from_records(pubsAlgorithmOverview)

def format_vertical_headers(df):
    """Display a dataframe with vertical column headers"""
    styles = [dict(selector="th", props=[('width', '40px')]),
              dict(selector="th.col_heading",
                   props=[("writing-mode", "vertical-rl"),
                          ('transform', 'rotateZ(180deg)'), 
                          ('vertical-align', 'top')])]
    return df.fillna('').style.set_table_styles(styles)

with pandas.option_context("display.max_rows", len(df.index)):
    display(format_vertical_headers(df).set_properties(subset=['Name', 'Journal'], **{'width': '150px'}))

,Name,Year,Citations,Type,Approach,Static Stability,Dynamic Stability,Planar Rotations,Verticalizations,Unload Priority,Family Groupings,Visibility,Concave Items,Load Bearing,Layer Packing,Delivery Time,Container Shape,Journal
0,"Paquay, Schyns, Limbourg",2016,83,MILP,Exact,$\checkmark$,$\checkmark$,$\checkmark$,$\checkmark$,,,,,$\checkmark$,,,$\checkmark$,International Transactions in Operational Research 23 (2016)
1,TS2Pack,2009,146,Tabu,Heuristic,,,,,,,,,,,,,European Journal of Operational Research 195 (2009)
2,"S. Martello, D. Pisinger, D. Vigo",2000,741,B&B,Exact,,,,,,,,,,$\checkmark$,,,Operations Research 48 (2000)
3,"Calzavara, Iori et al.",2021,0,(Constructive Heuristic/MILP)+GRASP,Heuristic,$\checkmark$,,$\checkmark$,,,$\checkmark$,$\checkmark$,,$\checkmark$,$\checkmark$,,,Annals of Operations Research (2021)
4,"Egeblad, Garavelli, Lisi, Pisinger",2012,71,Tree Search+ Local Serach+ Greedy Search,Heuristic,$\checkmark$,,$\checkmark$,$\checkmark$,,,,$\checkmark$,$\checkmark$,,,,European Journal of Operational Research 200 (2010)
5,"Alonso, Alvarez-Valdés, Parreño et al.",2020,5,GRASP,Heuristic,$\checkmark$,$\checkmark$,$\checkmark$,$\checkmark$,,,,,$\checkmark$,$\checkmark$,$\checkmark$,,4OR - A Quarterly Journal of Operations Research 18 (2020)
6,A Zudio et al.,2018,12,BRKGA+VND,Heuristic,,,$\checkmark$,$\checkmark$,,,,,,,,,Electronic Notes in Discrete Mathematics 66 (2018)


# Possible Moves

## Don't pack in Z
Given a packing $P$ composed of $S_z \in P$ closed support planes, where $z$ is is the base of the support plane. Given a proper $0 \le k \le H$, search for a small packed item $p_i \in S_{z'}$ where $ z-h \ge z' \ge z $.

Unpack every support plane $S_i$ with $ i \ge z' $, forbid the condition $p_i \in S_{z'}$ and repack all $p_j \in \bigcup\limits_{i=z'}^{z} S_i$ and end up with a packing $P'$

If $s(P) > s(P)$ then keep the new packing